In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

# Ref 1: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
# Ref 2: https://keras.io/getting-started/functional-api-guide/#multi-input-and-multi-output-models

import pandas as pd
import numpy as np

import cv2

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

from os import path
import os


def image_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized



def visualize(folders, model=''):
    if model != '':
        predition_csv = '{}_prediction.csv'.format(model)
        predition_video = '{}_visualization.mp4'.format(model)
    else:
        predition_csv = 'predictions.csv'
        predition_video = 'visualization.mp4'

    for folder in folders:
        video_path = folder + '/video.mp4'
        prediction_path = folder + '/predictions.csv'
        truth_path = folder + '/data_smooth.csv'
        out_video_path = folder + '/' + predition_video

        if not path.exists(folder):
            print("Wrong Folder! - " + folder)
            return
        elif not path.exists(video_path):
            print("No Video! - " + folder)
            return
        elif not path.exists(prediction_path):
            print("No Predictions! - " + folder)
            return

        prediction = pd.DataFrame(pd.read_csv(prediction_path, header=None).values)
        ground_truth = pd.DataFrame(pd.read_csv(truth_path, header=None).values)
        cap = cv2.VideoCapture(video_path)

        # Check if camera opened successfully
        if (cap.isOpened() == False):
            print("Error opening video stream or file")
        else:
            print("Successfully opened video!")

        frame_id = 0
        fig, axs = plt.subplots(2)
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        fig.subplots_adjust(hspace=0.5)

        acc_pred_points = [[], [], []]
        acc_truth_points = [[], [], []]

        # For preview purpose.
        preview_shape = (288, 864)  # Hard coded!
        car_size_2 = (10, 20)
        starting_point = (int(preview_shape[0] / 2), car_size_2[1])
        # Retrieved from data.
        v_truth = ground_truth.iloc[0, 0:2].tolist()
        v_pred = v_truth[:]
        acc_truth = [0.0, 0.0]
        acc_pred = [0.0, 0.0]

        pos_truth = [0, starting_point[1], ]
        pos_pred = pos_truth[:]
        pos_front = []

        out = None

        # Find the maximum absolute value for axis limitations.
        lim_x_truth = max(abs(ground_truth.iloc[:, -3].max()), abs(ground_truth.iloc[:, -3].min()))
        lim_y_truth = max(abs(ground_truth.iloc[:, -2].max()), abs(ground_truth.iloc[:, -2].min()))
        lim_x_pred = max(abs(prediction.iloc[:, 0].max()), abs(prediction.iloc[:, 1].min()))
        lim_y_pred = max(abs(prediction.iloc[:, 0].max()), abs(prediction.iloc[:, 1].min()))
        lim_acc = max(lim_x_truth, lim_y_truth, lim_x_pred, lim_y_pred)

        # Read until video is completed
        while (cap.isOpened() and frame_id < len(prediction) and frame_id < len(ground_truth)):
            # Capture frame-by-frame
            ret, frame = cap.read()
            if ret == True:
                frame_h, frame_w, frame_c = frame.shape

                dt = 0.1  # Scale for moving the vehicles
                pos_truth = [(pos_truth[i] + v_truth[i] * dt) for i in range(2)]
                pos_truth[0] = 0.0
                v_truth = ground_truth.iloc[frame_id, 0:2].tolist()

                d_front = ground_truth.iloc[frame_id, 3:5].tolist()[::-1]
                pos_front = [(pos_truth[i] + d_front[i]) for i in range(2)]

                pos_pred = [(pos_pred[i] + v_pred[i] * dt) for i in range(2)]
                # Caution: SIGN!!!
                v_pred = [(v_pred[i] + acc_pred[::-1][i] * dt) for i in range(2)]

                acc_pred = prediction.iloc[frame_id, 0:2].tolist()
                acc_truth = ground_truth.iloc[frame_id, -3:].tolist()

                for i in range(2):
                    # Caution: SIGN!!!
                    acc_pred_points[i].append(acc_pred[i])
                    acc_truth_points[i].append(acc_truth[i])

                axs[0].set_title("x acc")
                axs[1].set_title("y acc")
                for i in range(2):
                    axs[i].set_xlim(0, 200)
                    margin = 0.2
                    axs[i].set_ylim(-lim_acc * (1 + margin), lim_acc * (1 + margin))
                    axs[i].hlines(0, 0, 200, '0.5', lw=1, linestyles='dashed')
                    axs[i].plot(acc_pred_points[i], label="Prediction")
                    axs[i].plot(acc_truth_points[i], label="Ground Truth")
                    axs[i].legend(loc='upper right')

                canvas.draw()  # draw the canvas, cache the renderer
                fig_w, fig_h = fig.get_size_inches() * fig.get_dpi()
                fig_w, fig_h = (int(fig_w), int(fig_h))
                frame = image_resize(frame, width=fig_w)
                image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(fig_h, fig_w, 3)

                combined = cv2.hconcat([image, frame])
                display_scale = (0.5, 8)
                pred_preview = np.zeros((combined.shape[0], combined.shape[1], 3), np.uint8)
                cv2.line(pred_preview, (0, starting_point[0]),
                         (int(pred_preview.shape[1]), starting_point[0]), (100, 100, 100), 1)
                cv2.line(pred_preview, (display_scale[1] * starting_point[1], 0),
                         (display_scale[1] * starting_point[1], int(pred_preview.shape[0])), (100, 200, 100), 1)
                cv2.rectangle(pred_preview,
                              (int(display_scale[1] * pos_truth[1]) - car_size_2[1],
                               int(display_scale[0] * pos_truth[0]) - car_size_2[0] + int(preview_shape[0] / 2)),
                              (int(display_scale[1] * pos_truth[1]) + car_size_2[1],
                               int(display_scale[0] * pos_truth[0]) + car_size_2[0] + int(preview_shape[0] / 2)),
                              (200, 0, 0), 2)
                cv2.rectangle(pred_preview,
                              (int(display_scale[1] * pos_pred[1]) - car_size_2[1],
                               int(display_scale[0] * pos_pred[0]) - car_size_2[0] + int(preview_shape[0] / 2)),
                              (int(display_scale[1] * pos_pred[1]) + car_size_2[1],
                               int(display_scale[0] * pos_pred[0]) + car_size_2[0] + int(preview_shape[0] / 2)),
                              (0, 155, 232), 2)
                cv2.rectangle(pred_preview,
                              (int(display_scale[1] * pos_front[1]) - car_size_2[1],
                               int(display_scale[0] * pos_front[0]) - car_size_2[0] + int(preview_shape[0] / 2)),
                              (int(display_scale[1] * pos_front[1]) + car_size_2[1],
                               int(display_scale[0] * pos_front[0]) + car_size_2[0] + int(preview_shape[0] / 2)),
                              (0, 0, 255), 2)

                combined = cv2.vconcat([combined, pred_preview])
                comb_shape = combined.shape
                if out is None:
                    out = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'XVID'), 10,
                                          (comb_shape[1], comb_shape[0]))

                out.write(combined)
                canvas.renderer.clear()
                for i in range(2):
                    axs[i].clear()
                    if frame_id % 20 == 0:
                      print("[%s]: %.2f%% finished" % (folder, frame_id / 198.0 * 100.0))
            else:
                break
            frame_id += 1

        # When everything done, release the video capture object
        plt.close(fig)
        cap.release()
        out.release()

        print("[%s] finished" % (folder))


import glob

test_folders = glob.glob('/content/drive/My Drive/vis')
print(test_folders)
test_num = len(test_folders)
print('test folders num:', len(test_folders))

model_name = 'double_lstm_600_smooth'


visualize(test_folders, model=model_name)

['/content/drive/My Drive/vistwo']
test folders num: 1
Successfully opened video!
[/content/drive/My Drive/vistwo]: 0.00% finished
[/content/drive/My Drive/vistwo]: 0.00% finished
[/content/drive/My Drive/vistwo]: 10.10% finished
[/content/drive/My Drive/vistwo]: 10.10% finished
[/content/drive/My Drive/vistwo]: 20.20% finished
[/content/drive/My Drive/vistwo]: 20.20% finished
[/content/drive/My Drive/vistwo]: 30.30% finished
[/content/drive/My Drive/vistwo]: 30.30% finished
[/content/drive/My Drive/vistwo]: 40.40% finished
[/content/drive/My Drive/vistwo]: 40.40% finished
[/content/drive/My Drive/vistwo]: 50.51% finished
[/content/drive/My Drive/vistwo]: 50.51% finished
[/content/drive/My Drive/vistwo]: 60.61% finished
[/content/drive/My Drive/vistwo]: 60.61% finished
[/content/drive/My Drive/vistwo]: 70.71% finished
[/content/drive/My Drive/vistwo]: 70.71% finished
[/content/drive/My Drive/vistwo]: 80.81% finished
[/content/drive/My Drive/vistwo]: 80.81% finished
[/content/drive/My D